# Load squared exponential approximation precomputed in data/*.json

In [7]:
import json
import numpy as np

# Exponential kernel
path_exp = "data/exp.json"

with open(path_exp, "r") as f:
    squared_approx = json.load(f)

# available decomposition sizes
squared_approx.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'])

In [8]:
decomp_size = 16
a_coeff = squared_approx[str(decomp_size)]['a']
b_coeff = np.exp(squared_approx[str(decomp_size)]['b'])

## Compute all submatrices (on interval [0, 1])

In [9]:
from kl_decomposition import galerkin
comp_degree = 30
quad = 500
g = 4.0

A_all = []

for b in b_coeff:
    A_all.append(galerkin.assemble_duffy(f=b, degree=comp_degree, quad=quad, gx=g))

# Assemble block of A as operators for iterative eigenvalue decomposition

In [21]:
import numpy as np
from itertools import product
from functools import partial
from scipy.sparse.linalg import LinearOperator, eigsh             # SciPy ≥ 1.10

# --------------------------------------------------------------------------- #
# 0.  helper: "mode-dot" using einsum   [  X ← M @ X  along one axis  ]
# --------------------------------------------------------------------------- #


def _left_mult_axis(X, M, axis):
    """
    Return tensor Y whose 'axis'–th index is left-multiplied by matrix M.

    X   ... ndarray with ndim = n
    M   ... 2-D ndarray   (d_out, d_in)
    """
    # move the axis to the front, do the contraction, and move it back
    X_move = np.moveaxis(X, axis, 0)              # (d_in, …)
    Y = np.einsum('ij,j...->i...', M, X_move)  # einsum does the matmul
    return np.moveaxis(Y, 0, axis)                # (d_out, …)


# --------------------------------------------------------------------------- #
# 1.  split local matrices into even/odd dense blocks
# --------------------------------------------------------------------------- #
def _even_odd_blocks(A):
    idx_even = np.arange(0, A.shape[0], 2)
    idx_odd = np.arange(1, A.shape[0], 2)
    return (A[np.ix_(idx_even, idx_even)],
            A[np.ix_(idx_odd, idx_odd)])


def _preprocess_blocks(A_all):
    even_blocks, odd_blocks = [], []
    for A in A_all:
        E, O = _even_odd_blocks(A)
        even_blocks.append(E)
        odd_blocks.append(O)
    return even_blocks, odd_blocks


# --------------------------------------------------------------------------- #
# 2.  make a LinearOperator for ONE parity sector  p = (p₀,…,p_{n-1})
# --------------------------------------------------------------------------- #
def _make_block_operator(bits, even_blocks, odd_blocks, g_coeff):
    """
    bits         ... parity pattern, tuple of 0/1, length n
    even_blocks  ... list[ndarray]  (len = M)
    odd_blocks   ... list[ndarray]  (len = M)
    g_coeff      ... list[float]    (len = M)
    """
    n = len(bits)
    M = len(g_coeff)
    # dimension along each tensor mode
    d_even, d_odd = even_blocks[0].shape[0], odd_blocks[0].shape[0]
    dims = [d_even if b == 0 else d_odd for b in bits]
    size_block = int(np.prod(dims))

    # --- matvec ------------------------------------------------------------ #
    def matvec(x):
        X = x.reshape(dims)                  # vector → tensor
        Y_tot = np.zeros_like(X)
        for k in range(M):
            # pick the local block for *this* k and each axis
            mats = [even_blocks[k] if b == 0 else odd_blocks[k] for b in bits]
            T = X
            for axis, Mk in enumerate(mats):      # n mode-multiplications
                T = _left_mult_axis(T, Mk, axis)
            Y_tot += g_coeff[k] * T
        return Y_tot.ravel()                      # tensor → vector

    return LinearOperator((size_block, size_block),
                          matvec=matvec,
                          rmatvec=matvec,         # Hermitian ⇒ same action
                          dtype=even_blocks[0].dtype)


# --------------------------------------------------------------------------- #
# 3.  main routine
# --------------------------------------------------------------------------- #
def spectral_blocks(A_all, a_coeff, n, N_eval=6):
    """
    Parameters
    ----------
    A_all   : list of ndarrays – the M local checker-board matrices A_k
    a_coeff : list/array         – the g_k coefficients (same length M)
    n       : int                – tensor order  (= number of sites/dimensions)
    N_eval  : int                – largest N_eval eigenvalues per block

    Returns
    -------
    eigvals : dict {parity_tuple : ndarray (desc. sorted)}
    """
    even_blocks, odd_blocks = _preprocess_blocks(A_all)
    eigvals = {}

    for bits in product((0, 1), repeat=n):                # 2^n blocks
        op = _make_block_operator(bits, even_blocks, odd_blocks, a_coeff)
        k = min(N_eval, op.shape[0] - 1)                 # eigsh needs k < dim
        w, _ = eigsh(op, k=k, which='LA')                 # largest algebraic
        eigvals[bits] = np.sort(w)[::-1]                  # descending

    return eigvals



In [25]:

# --- spectra ----------------------------------------------------------- #
spectra = spectral_blocks(A_all, a_coeff, 2, N_eval=20)

# peek at one sector, e.g. p = (0,0,0,0)
print("Top-5 eigenvalues in the all-even block:")
spectra

Top-5 eigenvalues in the all-even block:


{(0,
  0): array([6.15427911e-01, 2.19059775e-02, 1.87530274e-02, 5.44974179e-03,
        3.20024907e-03, 3.09870844e-03, 1.78266907e-03, 1.65335691e-03,
        9.78963461e-04, 9.63089317e-04, 8.54191220e-04, 7.00471428e-04,
        6.78567441e-04, 4.52803501e-04, 4.33841284e-04, 4.11046569e-04,
        4.10562342e-04, 3.36091317e-04, 3.29822491e-04, 2.75633702e-04]),
 (0,
  1): array([0.08840408, 0.01059064, 0.00700638, 0.00297971, 0.00236935,
        0.00165656, 0.00121791, 0.00105512, 0.00083412, 0.0006225 ,
        0.00060367, 0.00055426, 0.0004701 , 0.0003767 , 0.00034939,
        0.00032572, 0.00028995, 0.00028713, 0.00024332, 0.00021856]),
 (1,
  0): array([0.08840408, 0.01059064, 0.00700638, 0.00297971, 0.00236935,
        0.00165656, 0.00121791, 0.00105512, 0.00083412, 0.0006225 ,
        0.00060367, 0.00055426, 0.0004701 , 0.0003767 , 0.00034939,
        0.00032572, 0.00028995, 0.00028713, 0.00024332, 0.00021856]),
 (1,
  1): array([0.028086  , 0.0049382 , 0.00440907, 0.0018

In [17]:
%load_ext line_profiler

In [20]:
%lprun -f spectral_decomposition -f build_all_blocks -f assemble_block_operator spectral_decomposition(A_all, a_coeff, 3, N=10)

Timer unit: 1e-09 s

Total time: 6.24518 s
File: /tmp/ipykernel_1568690/1999837027.py
Function: assemble_block_operator at line 49

Line #      Hits         Time  Per Hit   % Time  Line Contents
    49                                           def assemble_block_operator(E_blocks, O_blocks, g, parity_bits):
    50                                               """
    51                                               g:       list of g_k
    52                                               parity_bits: tuple/list of 0/1 of length n
    53                                               """
    54         8       4743.0    592.9      0.0      op = None
    55                                               # pick the right local block (E or O) for each site & k
    56       136     312309.0   2296.4      0.0      for k, (Ek, Ok) in enumerate(zip(E_blocks, O_blocks)):
    57       512     274892.0    536.9      0.0          local_factors = [Ek if bit == 0 else Ok for bit in parity_bits]
    58

In [16]:
eigvals_by_block = spectral_decomposition(A_all, a_coeff, 3, N=10)

# examine, e.g. block (0,0,0,0) – the “all-even” sector
print("largest eigenvalues in the (even,even,even,even) block:")
eigvals_by_block

largest eigenvalues in the (even,even,even,even) block:


{(0,
  0,
  0): array([0.53566146, 0.0132469 , 0.01017344, 0.01017344, 0.00221207,
        0.00202125, 0.00202125, 0.00106807, 0.00102784, 0.00102784]),
 (0,
  0,
  1): array([0.06410805, 0.00517812, 0.00462151, 0.00290281, 0.00135564,
        0.00096983, 0.00093513, 0.00071522, 0.00070419, 0.00046115]),
 (0,
  1,
  0): array([0.06410805, 0.00517812, 0.00462151, 0.00290281, 0.00135564,
        0.00096983, 0.00093513, 0.00071522, 0.00070419, 0.00046115]),
 (0,
  1,
  1): array([0.01646322, 0.00268574, 0.00184839, 0.00158865, 0.00071244,
        0.00066984, 0.0005309 , 0.00051754, 0.00034892, 0.00033357]),
 (1,
  0,
  0): array([0.06410805, 0.00517812, 0.00462151, 0.00290281, 0.00135564,
        0.00096983, 0.00093513, 0.00071522, 0.00070419, 0.00046115]),
 (1,
  0,
  1): array([0.01646322, 0.00268574, 0.00184839, 0.00158865, 0.00071244,
        0.00066984, 0.0005309 , 0.00051754, 0.00034892, 0.00033357]),
 (1,
  1,
  0): array([0.01646322, 0.00268574, 0.00184839, 0.00158865, 0.00071244,